In [1]:
#include<iostream>
#include<ostream>
#include "eigen-3.4.0/Eigen/Dense"

In [2]:
typedef Eigen::Matrix<double, Eigen::Dynamic, Eigen::Dynamic> MatrixXd;

Eigen::Vector2d a;
Eigen::Vector3d b;
Eigen::Vector4d c;

In [3]:
template <typename VectorType>
class EigenOperations {
public:
    static VectorType PerformOperations(const VectorType& v) {
        VectorType result = v;

        // Normalize the vector
        result.normalize();

        // Add the dot product with itself
        double dot = v.dot(v);
        for (int i = 0; i < result.size(); i++) {
            result[i] += dot;
        }

        // Subtract the squared norm
        double squaredNorm = v.squaredNorm();
        for (int i = 0; i < result.size(); i++) {
            result[i] -= squaredNorm;
        }

        return result;
    }
};


In [4]:
%%python

import cppyy

def display_eigen(vec, name):
    print("Vector : ", name, ", Dimensions : ", len(vec))
    for x in vec:
        print(x)

dynamic = cppyy.gbl.MatrixXd
a = cppyy.gbl.a
b = cppyy.gbl.b
c = cppyy.gbl.c

In [6]:
%%python

for i in range(len(a)):
    a[i] = i + 1

for i in range(len(b)):
    b[i] = i + 1

for i in range(len(c)):
    c[i] = i + 1

In [7]:
%%python

display_eigen(a, "A")
display_eigen(b, "B")
display_eigen(c, "C")

Vector :  A , Dimensions :  2
1.0
2.0
Vector :  B , Dimensions :  3
1.0
2.0
3.0
Vector :  C , Dimensions :  4
1.0
2.0
3.0
4.0


In [8]:
%%python

res2d = cppyy.gbl.EigenOperations["Eigen::Vector2d"].PerformOperations(a)
res3d = cppyy.gbl.EigenOperations["Eigen::Vector3d"].PerformOperations(b)
res4d = cppyy.gbl.EigenOperations["Eigen::Vector4d"].PerformOperations(c)

In [9]:
%%python

display_eigen(res2d, "A")
display_eigen(res3d, "B")
display_eigen(res4d, "C")

Vector :  A , Dimensions :  2
0.4472135954999583
0.8944271909999157
Vector :  B , Dimensions :  3
0.26726124191242384
0.5345224838248495
0.8017837257372733
Vector :  C , Dimensions :  4
0.1825741858350547
0.3651483716701094
0.5477225575051676
0.7302967433402223
